<a href="https://colab.research.google.com/github/Junjaee/Project_PinocchiOH/blob/main/finalProject_acorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [431]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


In [432]:
#데이터 불러오기 

from google.colab import files
uploaded =files.upload()

Saving navermovie_final.xlsx to navermovie_final (1).xlsx


In [458]:
#데이터 저장
df =pd.read_excel('navermovie_final.xlsx')
df.head(3)

,Unnamed: 0,MOVIE_CODE,MOVIE_TITLE,MOVIE_YEAR,MOVIE_RATING,MOVIE_GENRE,MOVIE_COUNTRY,MOVIE_RTIME,MOVIE_DIRECTOR,MOVIE_ACTOR,MOVIE_AGE10,MOVIE_AGE20,MOVIE_AGE30,MOVIE_AGE40,MOVIE_AGE50
0,0,10001,시네마 천국,1988,9.33,"드라마,멜로/로맨스","프랑스,이탈리아",124,쥬세페 토르나토레,"마르코 레오나르디,필립 느와레,자끄 페렝",9,47,25,6,12
1,1,10002,빽 투 더 퓨쳐,1985,9.42,"SF,코미디",미국,120,로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드,리 톰슨",2,61,18,16,2
2,2,10003,빽 투 더 퓨쳐 2,1989,9.24,"SF,코미디",미국,107,로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드",0,45,52,3,0


In [459]:
df.shape

(4289, 15)

In [460]:
# 칼럼중에서 추천알고리즘에 쓸꺼 고르기

columns = ['MOVIE_TITLE', 'MOVIE_GENRE','MOVIE_DIRECTOR','MOVIE_ACTOR']

In [461]:
df[columns].head(3)

,MOVIE_TITLE,MOVIE_GENRE,MOVIE_DIRECTOR,MOVIE_ACTOR
0,시네마 천국,"드라마,멜로/로맨스",쥬세페 토르나토레,"마르코 레오나르디,필립 느와레,자끄 페렝"
1,빽 투 더 퓨쳐,"SF,코미디",로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드,리 톰슨"
2,빽 투 더 퓨쳐 2,"SF,코미디",로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드"


In [462]:
#결측치 있는지 확인 

df[columns].isnull().values.any()

True

In [463]:
df[columns].shape

(4289, 4)

In [464]:
#결측치 있네..? 제거하기.. 
df1= df[columns].dropna()

In [465]:
#제거하니까 7개 없어짐 갠춘 ㅎㅎ
df1.shape

(4282, 4)

In [467]:
#df1 데이터 타입 보기 
df1.dtypes

MOVIE_TITLE       object
MOVIE_GENRE       object
MOVIE_DIRECTOR    object
MOVIE_ACTOR       object
dtype: object

In [468]:
#'MOVIE_TITLE', 'MOVIE_GENRE','MOVIE_DIRECTOR','MOVIE_ACTOR'

important_features = []
important_features.append(df1['MOVIE_TITLE']+' '+df1['MOVIE_GENRE']+' '+df1['MOVIE_DIRECTOR']+' '+df1['MOVIE_ACTOR'])


In [469]:
df_important = pd.DataFrame(important_features)


In [470]:
df_important = df_important.T

In [471]:
df_important

,0
0,"시네마 천국 드라마,멜로/로맨스 쥬세페 토르나토레 마르코 레오나르디,필립 느와레,자..."
1,"빽 투 더 퓨쳐 SF,코미디 로버트 저메키스 마이클 J. 폭스,크리스토퍼 로이드,리 톰슨"
2,"빽 투 더 퓨쳐 2 SF,코미디 로버트 저메키스 마이클 J. 폭스,크리스토퍼 로이드"
3,"금지된 장난 드라마,전쟁 르네 클레망 조르주 푸줄리,브리지트 포시"
4,"바람과 함께 사라지다 드라마,전쟁,멜로/로맨스 빅터 플레밍 클라크 게이블,비비안 리..."
...,...
4284,"썰 코미디,스릴러,미스터리,공포 황승재 김강현,찬희,김소라"
4285,"체르노빌 1986 액션,스릴러 다닐라 코즐로브스키 다닐라 코즐로브스키,오크사나 아킨..."
4286,"와인 패밀리 드라마,코미디,멜로/로맨스 숀 시스터나 조 판톨리아노,웬디 크로슨,폴라..."
4287,"괴기맨숀 공포 조바른 성준,김홍파,김보라"


In [472]:
#위에서 만든 칼럼모은거 넣을 원래 데이터에 칼럼으로 추가
#index 로 merge 하기

df= pd.merge(df, df_important, how='right', left_index=True, right_index=True)

In [475]:
df.head()

,Unnamed: 0,MOVIE_CODE,MOVIE_TITLE,MOVIE_YEAR,MOVIE_RATING,MOVIE_GENRE,MOVIE_COUNTRY,MOVIE_RTIME,MOVIE_DIRECTOR,MOVIE_ACTOR,MOVIE_AGE10,MOVIE_AGE20,MOVIE_AGE30,MOVIE_AGE40,MOVIE_AGE50,0
0,0,10001,시네마 천국,1988,9.33,"드라마,멜로/로맨스","프랑스,이탈리아",124,쥬세페 토르나토레,"마르코 레오나르디,필립 느와레,자끄 페렝",9,47,25,6,12,"시네마 천국 드라마,멜로/로맨스 쥬세페 토르나토레 마르코 레오나르디,필립 느와레,자..."
1,1,10002,빽 투 더 퓨쳐,1985,9.42,"SF,코미디",미국,120,로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드,리 톰슨",2,61,18,16,2,"빽 투 더 퓨쳐 SF,코미디 로버트 저메키스 마이클 J. 폭스,크리스토퍼 로이드,리 톰슨"
2,2,10003,빽 투 더 퓨쳐 2,1989,9.24,"SF,코미디",미국,107,로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드",0,45,52,3,0,"빽 투 더 퓨쳐 2 SF,코미디 로버트 저메키스 마이클 J. 폭스,크리스토퍼 로이드"
3,3,10014,금지된 장난,1952,8.86,"드라마,전쟁",프랑스,102,르네 클레망,"조르주 푸줄리,브리지트 포시",0,0,100,0,0,"금지된 장난 드라마,전쟁 르네 클레망 조르주 푸줄리,브리지트 포시"
4,4,10020,바람과 함께 사라지다,1939,9.23,"드라마,전쟁,멜로/로맨스",미국,230,빅터 플레밍,"클라크 게이블,비비안 리,레슬리 하워드",7,40,13,27,13,"바람과 함께 사라지다 드라마,전쟁,멜로/로맨스 빅터 플레밍 클라크 게이블,비비안 리..."


In [476]:
df = df.rename(columns={0:'important_features'})

In [477]:
df

,Unnamed: 0,MOVIE_CODE,MOVIE_TITLE,MOVIE_YEAR,MOVIE_RATING,MOVIE_GENRE,MOVIE_COUNTRY,MOVIE_RTIME,MOVIE_DIRECTOR,MOVIE_ACTOR,MOVIE_AGE10,MOVIE_AGE20,MOVIE_AGE30,MOVIE_AGE40,MOVIE_AGE50,important_features
0,0,10001,시네마 천국,1988,9.33,"드라마,멜로/로맨스","프랑스,이탈리아",124,쥬세페 토르나토레,"마르코 레오나르디,필립 느와레,자끄 페렝",9,47,25,6,12,"시네마 천국 드라마,멜로/로맨스 쥬세페 토르나토레 마르코 레오나르디,필립 느와레,자..."
1,1,10002,빽 투 더 퓨쳐,1985,9.42,"SF,코미디",미국,120,로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드,리 톰슨",2,61,18,16,2,"빽 투 더 퓨쳐 SF,코미디 로버트 저메키스 마이클 J. 폭스,크리스토퍼 로이드,리 톰슨"
2,2,10003,빽 투 더 퓨쳐 2,1989,9.24,"SF,코미디",미국,107,로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드",0,45,52,3,0,"빽 투 더 퓨쳐 2 SF,코미디 로버트 저메키스 마이클 J. 폭스,크리스토퍼 로이드"
3,3,10014,금지된 장난,1952,8.86,"드라마,전쟁",프랑스,102,르네 클레망,"조르주 푸줄리,브리지트 포시",0,0,100,0,0,"금지된 장난 드라마,전쟁 르네 클레망 조르주 푸줄리,브리지트 포시"
4,4,10020,바람과 함께 사라지다,1939,9.23,"드라마,전쟁,멜로/로맨스",미국,230,빅터 플레밍,"클라크 게이블,비비안 리,레슬리 하워드",7,40,13,27,13,"바람과 함께 사라지다 드라마,전쟁,멜로/로맨스 빅터 플레밍 클라크 게이블,비비안 리..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4284,58,204781,썰,2021,6.33,"코미디,스릴러,미스터리,공포",한국,84,황승재,"김강현,찬희,김소라",0,100,0,0,0,"썰 코미디,스릴러,미스터리,공포 황승재 김강현,찬희,김소라"
4285,59,204782,체르노빌 1986,2021,7.96,"액션,스릴러",러시아 연방,136,다닐라 코즐로브스키,"다닐라 코즐로브스키,오크사나 아킨쉬나,필리프 아브데예프",9,43,22,22,4,"체르노빌 1986 액션,스릴러 다닐라 코즐로브스키 다닐라 코즐로브스키,오크사나 아킨..."
4286,60,205285,와인 패밀리,2019,8.68,"드라마,코미디,멜로/로맨스","캐나다,이탈리아",94,숀 시스터나,"조 판톨리아노,웬디 크로슨,폴라 브랜카티",0,67,33,0,0,"와인 패밀리 드라마,코미디,멜로/로맨스 숀 시스터나 조 판톨리아노,웬디 크로슨,폴라..."
4287,61,205629,괴기맨숀,2021,6.21,공포,한국,106,조바른,"성준,김홍파,김보라",0,71,29,0,0,"괴기맨숀 공포 조바른 성준,김홍파,김보라"


In [478]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [479]:
# movie code 가 뭔가 숫자가 커서그런지.. 안되니까.. 따로 인덱스 따서 번호하나씩 정해쥼
df1 = df.reset_index()
df1
df = df1.rename(columns={'index':'MOVIE_N'})


In [480]:
df

,MOVIE_N,MOVIE_CODE,MOVIE_TITLE,MOVIE_YEAR,MOVIE_RATING,MOVIE_GENRE,MOVIE_COUNTRY,MOVIE_RTIME,MOVIE_DIRECTOR,MOVIE_ACTOR,MOVIE_AGE10,MOVIE_AGE20,MOVIE_AGE30,MOVIE_AGE40,MOVIE_AGE50,important_features
0,0,10001,시네마 천국,1988,9.33,"드라마,멜로/로맨스","프랑스,이탈리아",124,쥬세페 토르나토레,"마르코 레오나르디,필립 느와레,자끄 페렝",9,47,25,6,12,"시네마 천국 드라마,멜로/로맨스 쥬세페 토르나토레 마르코 레오나르디,필립 느와레,자..."
1,1,10002,빽 투 더 퓨쳐,1985,9.42,"SF,코미디",미국,120,로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드,리 톰슨",2,61,18,16,2,"빽 투 더 퓨쳐 SF,코미디 로버트 저메키스 마이클 J. 폭스,크리스토퍼 로이드,리 톰슨"
2,2,10003,빽 투 더 퓨쳐 2,1989,9.24,"SF,코미디",미국,107,로버트 저메키스,"마이클 J. 폭스,크리스토퍼 로이드",0,45,52,3,0,"빽 투 더 퓨쳐 2 SF,코미디 로버트 저메키스 마이클 J. 폭스,크리스토퍼 로이드"
3,3,10014,금지된 장난,1952,8.86,"드라마,전쟁",프랑스,102,르네 클레망,"조르주 푸줄리,브리지트 포시",0,0,100,0,0,"금지된 장난 드라마,전쟁 르네 클레망 조르주 푸줄리,브리지트 포시"
4,4,10020,바람과 함께 사라지다,1939,9.23,"드라마,전쟁,멜로/로맨스",미국,230,빅터 플레밍,"클라크 게이블,비비안 리,레슬리 하워드",7,40,13,27,13,"바람과 함께 사라지다 드라마,전쟁,멜로/로맨스 빅터 플레밍 클라크 게이블,비비안 리..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4277,4284,204781,썰,2021,6.33,"코미디,스릴러,미스터리,공포",한국,84,황승재,"김강현,찬희,김소라",0,100,0,0,0,"썰 코미디,스릴러,미스터리,공포 황승재 김강현,찬희,김소라"
4278,4285,204782,체르노빌 1986,2021,7.96,"액션,스릴러",러시아 연방,136,다닐라 코즐로브스키,"다닐라 코즐로브스키,오크사나 아킨쉬나,필리프 아브데예프",9,43,22,22,4,"체르노빌 1986 액션,스릴러 다닐라 코즐로브스키 다닐라 코즐로브스키,오크사나 아킨..."
4279,4286,205285,와인 패밀리,2019,8.68,"드라마,코미디,멜로/로맨스","캐나다,이탈리아",94,숀 시스터나,"조 판톨리아노,웬디 크로슨,폴라 브랜카티",0,67,33,0,0,"와인 패밀리 드라마,코미디,멜로/로맨스 숀 시스터나 조 판톨리아노,웬디 크로슨,폴라..."
4280,4287,205629,괴기맨숀,2021,6.21,공포,한국,106,조바른,"성준,김홍파,김보라",0,71,29,0,0,"괴기맨숀 공포 조바른 성준,김홍파,김보라"


In [481]:
 # 문자 숫자로 바꾸기  (convert the text to a matrix of token counts - > sklearn)

 cm = CountVectorizer().fit_transform(df['important_features'])

In [482]:
# cosine similarity matirix from the count matrix

cs = cosine_similarity(cm)

#print cosine similarity matrix

print(cs)

[[1.         0.         0.         ... 0.24019223 0.         0.        ]
 [0.         1.         0.9486833  ... 0.09128709 0.         0.        ]
 [0.         0.9486833  1.         ... 0.09622504 0.         0.        ]
 ...
 [0.24019223 0.09128709 0.09622504 ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [483]:
#get the shape of the cosine similarity matrix

cs.shape

(4282, 4282)

In [484]:
# 추천받기전 내가 조아하는 영화를 title로 
title = "피노키오의 모험"

#find the movie's id
#피노키오 movie_n 칼럼에서 몇번쨰인지
movie_num =df[df.MOVIE_TITLE == title]['MOVIE_N'].values[0]

In [486]:
movie_num

1633

In [487]:
#create a list of enumerations for the similarity score

scores = list(enumerate(cs[movie_num]))

In [488]:
#sort the list 
# x[1] - > similarity score 

sorted_scores = sorted(scores, key = lambda x:x[1], reverse = True)
sorted_scores = sorted_scores[1:] 
#자기자신 포함 안하려고 1은 제외하고 

In [489]:
#print sorted scores

print(sorted_scores)

#()

[(3477, 0.4), (419, 0.33541019662496846), (1352, 0.33541019662496846), (340, 0.31622776601683794), (3196, 0.31622776601683794), (3266, 0.31622776601683794), (332, 0.30000000000000004), (957, 0.30000000000000004), (1973, 0.30000000000000004), (3032, 0.2860387767736777), (3170, 0.2860387767736777), (3968, 0.2860387767736777), (1345, 0.27386127875258315), (1536, 0.27386127875258315), (3106, 0.27386127875258315), (750, 0.2631174057921088), (3021, 0.2631174057921088), (3067, 0.2631174057921088), (1448, 0.25819888974716115), (2224, 0.25819888974716115), (2280, 0.25819888974716115), (3123, 0.25819888974716115), (3339, 0.25819888974716115), (3400, 0.25819888974716115), (3901, 0.25819888974716115), (262, 0.253546276418555), (475, 0.253546276418555), (1667, 0.253546276418555), (1064, 0.2449489742783178), (1308, 0.2449489742783178), (1317, 0.2449489742783178), (1583, 0.2449489742783178), (3176, 0.2449489742783178), (685, 0.23904572186687872), (1207, 0.23904572186687872), (1399, 0.2390457218668787

In [490]:
#일단 7개 보구, 오른쪽이 similarity 니까.. 나중에 percentage로 해서 말해주는데 사용해도 괜찮을듯...?
sorted_scores[:7]

[(3477, 0.4),
 (419, 0.33541019662496846),
 (1352, 0.33541019662496846),
 (340, 0.31622776601683794),
 (3196, 0.31622776601683794),
 (3266, 0.31622776601683794),
 (332, 0.30000000000000004)]

In [491]:
j = 0

print( title, "을 선택한 당신에게 추천하는 7가지 영화")

for item in sorted_scores:
  movie_title = df[df.MOVIE_N == item[0]]['MOVIE_TITLE'].values[0]
  print(j +1, movie_title)
  j = j+1
  if j>6:
    break

피노키오의 모험 을 선택한 당신에게 추천하는 7가지 영화
1 키코리키: 시간여행
2 엑시덴탈 러브
3 일만명의 성자들
4 러브 액츄얼리
5 버든: 세상을 바꾸는 힘
6 극장판 요괴워치 섀도사이드: 도깨비왕의 부활
7 8 마일
